In [18]:
import os
import numpy as np

import torch
import torchvision

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import matplotlib.pyplot as plt

import torchvision.transforms.functional as TF

%matplotlib inline

In [19]:
# image processing package
from PIL import Image, ImageOps
import numpy as np

In [20]:
from unet_pytorch import UNet

## Model Architecture

In [ ]:
# Not WORKING. 
# You have to initialize conv layers in init functiion otherwise forward function keep initializing it at every step.
# Can not do like Keras in Pytorch.
# class LungSegmentationNet(nn.Module):
#     def __init__(self, depth=5, root_filter=64):
#         super(LungSegmentationNet, self).__init__()
#         self.depth = depth
#         self.root_filter = root_filter
#         print("depth: {}, root filter: {}".format(self.depth, self.root_filter))
    
#     # Down sampling
#     def forward(self, x):
#         long_connection_store = {}
#         input_channels = 1
#         print("UP SAMPLING")
#         for i in range(self.depth):
#             out_channel = 2**i * self.root_filter
#             print(out_channel)
#             x = F.relu(nn.Conv2d(input_channels, out_channel, 3, padding=1)(x))
#             x = F.relu(nn.Conv2d(out_channel, out_channel, 3, padding=1)(x))

#             if i<self.depth-1:
#                 long_connection_store[str(i)] = x
#                 x = F.max_pool2d(x, kernel_size=2)
#             input_channels = out_channel

#         print("x.shape: {}".format(x.shape))
#         print("DOWN SAMPLING")
        
#         for i in range(self.depth - 2, -1, -1):
#             out_channel = 2**(i) * self.root_filter
#             print("input_channels: {}".format(input_channels))
#             print("out_channel: {}".format(out_channel))
            
#             # long connection from down sampling path.
#             long_connection = long_connection_store[str(i)]
#             print("long_connection shape: {}".format(long_connection.shape))
            
#             up1 = nn.Upsample(scale_factor=2)(x)
#             print("up1 shape: {}".format(up1.shape))
            
#             up_conv1 = F.relu(nn.Conv2d(input_channels, out_channel, 3, padding=1)(up1))
#             print("upsamplingConv : {}".format(up_conv1.shape))
            
#             up_conc = torch.cat((up_conv1, long_connection), dim=1)
#             print("up_conc : {}\n".format(up_conc.shape))
            
#             #  Convolutions
#             up_conv1 = F.relu(nn.Conv2d(input_channels, out_channel, 3, padding=1)(up_conc))
#             up_conv2 = F.relu(nn.Conv2d(out_channel, out_channel, 3, padding=1)(up_conv1))

#             x = up_conv2
#             input_channels = out_channel
#         print("FINAL shape: {}".format(x.shape))
#         x = F.sigmoid(nn.Conv2d(input_channels, 1, 1, padding=0)(x))
#         print("FINAL shape: {}".format(x.shape))
#         return x
    


In [ ]:
# unet = LungSegmentationNet(5, 64)
# list(unet.parameters())

In [ ]:
model = UNet(in_channels=1)

In [ ]:
print(model)

In [ ]:
batch_size = 5
channels = 1
height = 256
width = 256

In [ ]:
image = torch.randn(batch_size, channels, height, width)

In [ ]:
image.shape

In [ ]:
output = model(image)

In [ ]:
output

In [ ]:
# from torchvision import models
# model = models.vgg16()
# print(model)

# Dataset Generator

In [21]:
# PyTorch is channel first instead of channel last as in Keras
class LungSegmentationDataGen(Dataset):
    def __init__(self, dataset, root_dir, args, transforms=None):
        self.dataset = dataset
        self.root_dir = root_dir
        self.args = args
    
    def __len__(self):
        return len(self.dataset)
    
    def preprocess_image(self, path):
        im = Image.open(path)
        im = ImageOps.grayscale(im)
        im = im.resize((self.args.height, self.args.width))  # parameterize height and width
        img = np.array(im)
        img = img/255.
        img = np.expand_dims(img, axis=0)
        return img
    
    def __getitem__(self, idx):
        sample = self.dataset[idx]
        img_name, mask_name = sample.rstrip().split(",")
        
        image = self.preprocess_image(os.path.join(self.root_dir, "images", img_name.strip()))
        mask = self.preprocess_image(os.path.join(self.root_dir, "masks", mask_name.strip()))
#         print(image.dtype, image.shape)
#         print(mask.dtype, mask.shape)
        # Use albumentation package for augmenting images and mask with same transformations
        
        # numpy uses float64 as their default type, so call float() on these tensors before passing them to the TensorDataset 
        return torch.from_numpy(image).float(), torch.from_numpy(mask).float()
        

In [22]:
def dice_score(y_true, y_pred):
    smooth = 1
    y_true_flatten = y_true.view(-1)
    y_pred_flatten = y_pred.view(-1)
    intersection = 2 * torch.sum(y_true_flatten * y_pred_flatten)
    union = torch.sum(y_true_flatten) + torch.sum(y_pred_flatten)
    dice_score = (intersection + smooth)/(union + smooth)
    return dice_score

In [23]:
def dice_loss(y_true, y_pred):
    return 1 - dice_score(y_true, y_pred)

### Training steps
1. Define the train and validation set data loaders
2. Find the device
3. Create the model object and put it in device
4. Define loss function
5. Define the optimizer
6. Training Loop
        6.1 Epochs for-loop
            6.2 Phase for-loop (Train and Val) ["train", "val"]
                6.3 Dataloader loop i.e. loop over batches of the dataset
                    6.4 Get the batch data and put the data in device. Below steps are going over a single batch
                    6.5 call optimizer.zero_grad()
                    6.6 Set grad enabled scope with statement(True for train and False for val)
                        6.7 pass the input data through model 
                        6.8 Calculate the loss
                        6.9 If the phase is train then call loss.backward() and optimizer.step() and Also you can collect the loss in a list
                        6.10 If the phase is val, then you can store the loss or something else that makes sense
                    6.11 If phase is train and print loss summary after certain number of steps.
                6.12 If the phase is val, you can calculate and store best_acc(for any metric you want to use) and current_acc
                     and based on the comparison between best and current accuracy save the model. HERE whole data loop is finished. 

In [24]:
class Args(object):
    def __init__(self):
        self.batch_size = 2
        self.model_depth = 5
        self.width = 256
        self.root_filter_size = 32
        self.epochs = 100
        self.height = 256
        self.weights = "trained_model"

In [25]:
def train(args):
    dataset = open("dataset.csv", "r").readlines()
    train_set = dataset[:600]
    val_set = dataset[600:]
    root_dir = root_dir = "data/Lung_Segmentation/"

    train_data = LungSegmentationDataGen(train_set, root_dir, args)
    val_data = LungSegmentationDataGen(val_set, root_dir, args)

    train_dataloader = DataLoader(train_data, batch_size=5, shuffle=True, num_workers=4)

    val_dataloader = DataLoader(val_data, batch_size=5, shuffle=True, num_workers=4)
    
    dataloaders = {
        "train": train_dataloader,
        "val": val_dataloader
    }

    dataset_sizes = {"train": len(train_set), "val": len(val_set)}

    print("dataset_sizes: {}".format(dataset_sizes))

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    model = UNet(in_channels=1)
    #print("MODEL: {}".format(model))
    model = model.to(device)
    
    #optimizer = optim.Adam(unet.parameters(), lr=args.lr)
    optimizer = optim.Adam(model.parameters())
    
    loss_train = []
    loss_valid = []
    
    current_mean_dsc = 0.0
    best_validation_dsc = 0.0

    step = 0
    epochs = args.epochs
    for epoch in range(epochs):
        print('Epoch {}/{}'.format(epoch, epochs - 1))
        print('-' * 10)
        dice_score_list = []
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode
            
            # Iterate over data.
            for i, data in enumerate(dataloaders[phase]):
                inputs, y_true = data
                inputs = inputs.to(device)
                y_true = y_true.to(device)
                
                # zero the parameter gradients
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    y_pred = model(inputs)
                    
                    loss = dice_loss(y_true, y_pred)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        #print("step: {}, train_loss: {}".format(i, loss))
                        loss_train.append(loss.item())
                        
                        # calculate the gradients based on loss
                        loss.backward()
                        
                        # update the weights
                        optimizer.step()
                        
                    if phase == "val":
                        loss_valid.append(loss.item())
                        dsc = dice_score(y_true, y_pred)
                        print("step: {}, val_loss: {}, val dice_score: {}".format(i, loss, dsc))
                        dice_score_list.append(dsc.detach().numpy())
                        
                        
                if phase == "train" and (i + 1) % 10 == 0:
                    print("step:{}, train_loss: {}".format(i+1, np.mean(loss_train)))
                    loss_train = []
            if phase == "val":
                print("mean val_loss: {}".format(np.mean(loss_valid)))
                loss_valid = []
                current_mean_dsc = np.mean(dice_score_list)
                print("validation set dice_score: {}".format(current_mean_dsc))
                if current_mean_dsc > best_validation_dsc:
                    best_validation_dsc = current_mean_dsc
                    print("best dice_score on val set: {}".format(best_validation_dsc))
                    torch.save(model.state_dict(), os.path.join(args.weights, "unet_{0:.2f}.pt".format(best_validation_dsc)))
                

In [26]:
args = Args()

In [ ]:
train(args)

dataset_sizes: {'train': 600, 'val': 104}
Epoch 0/99
----------
PHASE: train
step:10, train_loss: 0.5429947018623352
step:20, train_loss: 0.4225769817829132
step:30, train_loss: 0.389515346288681
step:40, train_loss: 0.3467102885246277
step:50, train_loss: 0.32322834730148314
step:60, train_loss: 0.3147452771663666
step:70, train_loss: 0.2739475131034851
step:80, train_loss: 0.2522972285747528
step:90, train_loss: 0.23953257799148558
step:100, train_loss: 0.22505950331687927
step:110, train_loss: 0.19935805201530457
step:120, train_loss: 0.17690134048461914
PHASE: val
step: 0, val_loss: 0.21154260635375977, val dice_score: 0.7884573936462402
step: 1, val_loss: 0.2014142870903015, val dice_score: 0.7985857129096985
step: 2, val_loss: 0.21596616506576538, val dice_score: 0.7840338349342346
step: 3, val_loss: 0.1701037883758545, val dice_score: 0.8298962116241455
step: 4, val_loss: 0.19283920526504517, val dice_score: 0.8071607947349548
step: 5, val_loss: 0.20474237203598022, val dice_sco

In [ ]:
# np.mean([np.array(10, dtype = np.float32), np.array(20, dtype = np.float32), np.array(30, dtype = np.float32)])

## Reading data 

In [ ]:
import os
root_dir = "data/Lung_Segmentation/"

In [ ]:
# MCUCXR_0001_0.png
# CHNCXR_0001_0.png --- CHNCXR_0001_0_mask.png

In [ ]:
images_list = os.listdir(os.path.join(root_dir, "images"))
masks_list = os.listdir(os.path.join(root_dir, "masks"))

In [ ]:
f = open("dataset.csv", "w")
count = 0
for img in images_list:
    img_name = img.split(".png")[0]
    if img.startswith("CHNCXR"):
        mask_name = "{}_mask.png".format(img_name)
    elif img.startswith("MCUCXR"):
        mask_name = img
    if os.path.exists(os.path.join(root_dir, "masks", mask_name)):
        f.write("{}, {}\n".format(img, mask_name))
        count +=1
    
f.close()      

In [ ]:
dataset = open("dataset.csv", "r").readlines()

In [ ]:
len(dataset)